# **Hot Hand Phenomenon**

Use shot log information in the 2014-2015 NBA season to test the hot hand phenonenon

## Part 1 - Data Preparation and Exploration

In [ ]:

# Due to the configuration of the base Jupter image, the following imports are required for the regressions in the assignment to report the correct metrics

import sys
!{sys.executable} -m pip uninstall statsmodels --yes
!{sys.executable} -m pip uninstall numpy --yes
!{sys.executable} -m pip uninstall pandas --yes
!{sys.executable} -m pip uninstall patsy --yes
!{sys.executable} -m pip install numpy==1.17
!{sys.executable} -m pip install pandas==1.0
!{sys.executable} -m pip install patsy==0.5.2
!{sys.executable} -m pip install statsmodels==0.11.1

In [ ]:
#Import Libraries

import pandas as pd
import datetime as dt
import scipy.stats as sp
import numpy as np
import statsmodels.formula.api as sm
pd.set_option('display.max_columns', None)

In [ ]:
# Import Shotlog_14_15 and Player_Stats Datasets

Shotlog_1415=pd.read_csv("Assignment Data/Week 6/Shotlog_14_15.csv")
Player_Stats=pd.read_csv("Assignment Data/Week 6/Player_Stats_14_15.csv")
display(Shotlog_1415)

,game_id,date,match,home_team,away_team,home_away,result,final_margin,shot_number,quarter,game_clock,shot_clock,dribbles,touch_time,shot_dist,points,current_shot_outcome,closest_defender,closest_defender_id,closest_def_dist,current_shot_hit,points_earned,shoot_player,player_id,average_hit,shot_count,shot_per_game
0,21400280,5-Dec-14,ATL @ BKN,BKN,ATL,A,W,23,1,1,11:23,5.6,0,1.2,19.6,2,made,"Lopez, Brook",201572,6.6,1,2,al horford,201143,0.541259,715,10
1,21400280,5-Dec-14,ATL @ BKN,BKN,ATL,A,W,23,2,1,10:48,5.6,0,1.1,14.5,2,missed,"Lopez, Brook",201572,5.6,0,0,al horford,201143,0.541259,715,10
2,21400280,5-Dec-14,ATL @ BKN,BKN,ATL,A,W,23,3,1,9:40,11.0,0,0.4,5.8,2,missed,"Lopez, Brook",201572,4.7,0,0,al horford,201143,0.541259,715,10
3,21400280,5-Dec-14,ATL @ BKN,BKN,ATL,A,W,23,4,1,7:09,13.5,0,0.8,20.8,2,missed,"Lopez, Brook",201572,5.8,0,0,al horford,201143,0.541259,715,10
4,21400280,5-Dec-14,ATL @ BKN,BKN,ATL,A,W,23,5,2,5:49,12.9,0,0.9,20.1,2,missed,"Lopez, Brook",201572,6.4,0,0,al horford,201143,0.541259,715,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128064,21400350,14-Dec-14,WAS vs. UTA,WAS,UTA,H,W,9,11,3,2:59,21.1,3,4.0,1.7,2,made,"Burke, Trey",203504,4.7,1,2,john wall,202322,0.448513,874,15
128065,21400350,14-Dec-14,WAS vs. UTA,WAS,UTA,H,W,9,12,3,0:43,6.1,2,2.2,17.8,2,made,"Exum, Dante",203957,3.4,1,2,john wall,202322,0.448513,874,15
128066,21400350,14-Dec-14,WAS vs. UTA,WAS,UTA,H,W,9,13,4,2:18,4.3,0,0.0,3.2,2,missed,"Kanter, Enes",202683,1.2,0,0,john wall,202322,0.448513,874,15
128067,21400350,14-Dec-14,WAS vs. UTA,WAS,UTA,H,W,9,14,4,2:18,4.3,0,0.6,3.3,2,made,"Kanter, Enes",202683,1.4,1,2,john wall,202322,0.448513,874,15


In [ ]:
Shotlog_1415.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128069 entries, 0 to 128068
Data columns (total 30 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   game_id               128069 non-null  int64         
 1   date                  128069 non-null  datetime64[ns]
 2   match                 128069 non-null  object        
 3   home_team             128069 non-null  object        
 4   away_team             128069 non-null  object        
 5   home_away             128069 non-null  object        
 6   result                128069 non-null  object        
 7   final_margin          128069 non-null  int64         
 8   shot_number           128069 non-null  int64         
 9   quarter               128069 non-null  int64         
 10  game_clock            128069 non-null  object        
 11  shot_clock            122502 non-null  float64       
 12  dribbles              128069 non-null  int64         
 13 

In [ ]:
Player_Stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281 entries, 0 to 280
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   shoot_player  281 non-null    object 
 1   average_hit   281 non-null    float64
dtypes: float64(1), object(1)
memory usage: 4.5+ KB


In [ ]:
# Convert the “date” variable to a date type variable and calculate summary statistics for the “shot_clock” variable.


In [ ]:
import datetime as dt
Shotlog_1415['date']=pd.to_datetime(Shotlog_1415['date'])

In [ ]:
Shotlog_1415['shot_clock'].describe()

count    122502.000000
mean         12.453344
std           5.763265
min           0.000000
25%           8.200000
50%          12.300000
75%          16.675000
max          24.000000
Name: shot_clock, dtype: float64

In [ ]:
#Create a lagged variable “lag_shot_hit” to indicate the result of the previous shot by the same player at the same game.


Shotlog_1415['lag_shot_hit']=Shotlog_1415.sort_values(by=['quarter','game_clock'], ascending=[True, True]).groupby(['shoot_player','game_id'])['current_shot_hit'].shift(1)
Shotlog_1415.head()


,game_id,date,match,home_team,away_team,home_away,result,final_margin,shot_number,quarter,game_clock,shot_clock,dribbles,touch_time,shot_dist,points,current_shot_outcome,closest_defender,closest_defender_id,closest_def_dist,current_shot_hit,points_earned,shoot_player,player_id,average_hit,shot_count,shot_per_game,lag_shot_hit
0,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,1,1,11:23,5.6,0,1.2,19.6,2,made,"Lopez, Brook",201572,6.6,1,2,al horford,201143,0.541259,715,10,0.0
1,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,2,1,10:48,5.6,0,1.1,14.5,2,missed,"Lopez, Brook",201572,5.6,0,0,al horford,201143,0.541259,715,10,NaN
2,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,3,1,9:40,11.0,0,0.4,5.8,2,missed,"Lopez, Brook",201572,4.7,0,0,al horford,201143,0.541259,715,10,0.0
3,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,4,1,7:09,13.5,0,0.8,20.8,2,missed,"Lopez, Brook",201572,5.8,0,0,al horford,201143,0.541259,715,10,1.0
4,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,5,2,5:49,12.9,0,0.9,20.1,2,missed,"Lopez, Brook",201572,6.4,0,0,al horford,201143,0.541259,715,10,0.0


In [ ]:
#Create a variable “error” to indicate the prediction error for each shot and a variable “lagerror” for the prediction error for the previous shot. The “error” variable is defined as the difference between the outcome of the current shot and the average success rate (“average_hit”) and the “lagerror” variable is defined as the difference between the outcome of the previous shot and the average success rate.


Shotlog_1415['error']=Shotlog_1415['current_shot_hit']-Shotlog_1415['average_hit']
Shotlog_1415['lagerror']=Shotlog_1415['lag_shot_hit']-Shotlog_1415['average_hit']


In [ ]:
#Calculate summary statistics for the “error” and “lagerror” variables.

Shotlog_1415['error'].describe()

count    1.280690e+05
mean    -5.770049e-18
std      4.949640e-01
min     -7.124682e-01
25%     -4.491979e-01
50%     -3.850837e-01
75%      5.395973e-01
max      6.914894e-01
Name: error, dtype: float64

In [ ]:
Shotlog_1415['lagerror'].describe()

count    113726.000000
mean          0.002821
std           0.495594
min          -0.712468
25%          -0.449198
50%          -0.385084
75%           0.541071
max           0.691489
Name: lagerror, dtype: float64

## Part 2 - Conditional Probability and Autocorrelation

In [ ]:
#Create a dummy variable “conse_shot” that indicates a player made consecutive shots.
Shotlog_1415['conse_shot'] = np.where((Shotlog_1415['current_shot_hit']==1)&(Shotlog_1415['lag_shot_hit']==1), 1, 0)
Shotlog_1415.head()

,game_id,date,match,home_team,away_team,home_away,result,final_margin,shot_number,quarter,game_clock,shot_clock,dribbles,touch_time,shot_dist,points,current_shot_outcome,closest_defender,closest_defender_id,closest_def_dist,current_shot_hit,points_earned,shoot_player,player_id,average_hit,shot_count,shot_per_game,lag_shot_hit,error,lagerror,conse_shot
0,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,1,1,11:23,5.6,0,1.2,19.6,2,made,"Lopez, Brook",201572,6.6,1,2,al horford,201143,0.541259,715,10,0.0,0.458741,-0.541259,0
1,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,2,1,10:48,5.6,0,1.1,14.5,2,missed,"Lopez, Brook",201572,5.6,0,0,al horford,201143,0.541259,715,10,NaN,-0.541259,NaN,0
2,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,3,1,9:40,11.0,0,0.4,5.8,2,missed,"Lopez, Brook",201572,4.7,0,0,al horford,201143,0.541259,715,10,0.0,-0.541259,-0.541259,0
3,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,4,1,7:09,13.5,0,0.8,20.8,2,missed,"Lopez, Brook",201572,5.8,0,0,al horford,201143,0.541259,715,10,1.0,-0.541259,0.458741,0
4,21400280,2014-12-05,ATL @ BKN,BKN,ATL,A,W,23,5,2,5:49,12.9,0,0.9,20.1,2,missed,"Lopez, Brook",201572,6.4,0,0,al horford,201143,0.541259,715,10,0.0,-0.541259,-0.541259,0


In [ ]:
#Create a dataframe “Player_Prob” for the probability of making the previous shot and the joint probability for making both the previous and current shots. Name the probability of making the previous shot “average_lag_hit” and the probability of making both shots “conse_shot_hit.”
Player_Prob=Shotlog_1415.groupby(['shoot_player'])['conse_shot','lag_shot_hit'].mean()
Player_Prob=Player_Prob.reset_index()
Player_Prob.rename(columns={'lag_shot_hit':'average_lag_hit'}, inplace=True)
Player_Prob.head()


,shoot_player,conse_shot,average_lag_hit
0,aaron brooks,0.160428,0.418000
1,aaron gordon,0.192308,0.519481
2,al farouq aminu,0.158915,0.450980
3,al horford,0.269930,0.534954
4,al jefferson,0.202500,0.476000


In [ ]:
#In the “Player_Prob” dataframe, calculate the conditional probability “conditional_prob” for a player to make a shot given that he made the previous shot.
Player_Prob['conditional_prob']=Player_Prob['conse_shot']/Player_Prob['average_lag_hit']
Player_Prob.head()


,shoot_player,conse_shot,average_lag_hit,conditional_prob
0,aaron brooks,0.160428,0.418000,0.383799
1,aaron gordon,0.192308,0.519481,0.370192
2,al farouq aminu,0.158915,0.450980,0.352376
3,al horford,0.269930,0.534954,0.504585
4,al jefferson,0.202500,0.476000,0.425420


In [ ]:
#Merge the “Player_Prob” dataframe into the “Player_Stats” dataframe.
Player_Stats=pd.merge(Player_Prob, Player_Stats, on=['shoot_player'])
Player_Stats.head(10)

,shoot_player,conse_shot,average_lag_hit,conditional_prob,average_hit
0,aaron brooks,0.160428,0.418000,0.383799,0.415330
1,aaron gordon,0.192308,0.519481,0.370192,0.528846
2,al farouq aminu,0.158915,0.450980,0.352376,0.430233
3,al horford,0.269930,0.534954,0.504585,0.541259
4,al jefferson,0.202500,0.476000,0.425420,0.477500
5,alan anderson,0.175074,0.444444,0.393917,0.433234
6,alan crabbe,0.085106,0.444444,0.191489,0.425532
7,alex len,0.247492,0.547718,0.451860,0.528428
8,alexis ajinca,0.270142,0.598802,0.451137,0.597156
9,alonzo gee,0.152174,0.441176,0.344928,0.478261


In [ ]:
#Calculate summary statistics for the probability for a player to make a shot (“average_hit”) and the conditional probability for a player to make a shot given that he made the previous one (“conditional_prob”) and the probability of players making consecutive shots (“conse_shot_hit”).
Player_Stats['average_hit'].describe()


count    281.000000
mean       0.451545
std        0.059392
min        0.308511
25%        0.413223
50%        0.446078
75%        0.480480
max        0.712468
Name: average_hit, dtype: float64

In [ ]:
Player_Stats['conditional_prob'].describe()

count    281.000000
mean       0.380014
std        0.065066
min        0.186861
25%        0.344705
50%        0.384262
75%        0.423983
max        0.619830
Name: conditional_prob, dtype: float64

In [ ]:
Player_Stats['conse_shot'].describe()

count    281.000000
mean       0.175459
std        0.048954
min        0.061905
25%        0.142857
50%        0.170455
75%        0.198777
max        0.432570
Name: conse_shot, dtype: float64

In [ ]:
#Perform a t-test for the statistical significance on the difference between conditional probability and unconditional probability of making a shot.
sp.stats.ttest_ind(Player_Stats['conditional_prob'], Player_Stats['average_hit'])


Ttest_indResult(statistic=-13.611035692139202, pvalue=1.1858287648316392e-36)

In [ ]:
#Calculate the first order autocorrelation coefficient on making a shot (correlation coefficient between making the current shot and the previous shot) for the entire shotlog dataset.
Shotlog_1415['current_shot_hit'].corr(Shotlog_1415['lag_shot_hit'])


-0.00938408791603196

In [ ]:
#Calculate the first order autocorrelation coefficient on making a shot for each player. Display the top ten players with the highest first order autocorrelation coefficient.
Shotlog_1415.groupby('shoot_player')[['current_shot_hit','lag_shot_hit']].corr().head(10)


current_shot_hit  lag_shot_hit
shoot_player                                                    
aaron brooks    current_shot_hit          1.000000      0.007921
                lag_shot_hit              0.007921      1.000000
aaron gordon    current_shot_hit          1.000000     -0.094916
                lag_shot_hit             -0.094916      1.000000
al farouq aminu current_shot_hit          1.000000      0.062406
                lag_shot_hit              0.062406      1.000000
al horford      current_shot_hit          1.000000      0.009093
                lag_shot_hit              0.009093      1.000000
al jefferson    current_shot_hit          1.000000     -0.042402
                lag_shot_hit             -0.042402      1.000000

In [ ]:
Autocorr_Hit=Shotlog_1415.groupby('shoot_player')[['current_shot_hit','lag_shot_hit']].corr().unstack()
Autocorr_Hit.head()

current_shot_hit                  lag_shot_hit             
                current_shot_hit lag_shot_hit current_shot_hit lag_shot_hit
shoot_player                                                               
aaron brooks                 1.0     0.007921         0.007921          1.0
aaron gordon                 1.0    -0.094916        -0.094916          1.0
al farouq aminu              1.0     0.062406         0.062406          1.0
al horford                   1.0     0.009093         0.009093          1.0
al jefferson                 1.0    -0.042402        -0.042402          1.0

In [ ]:
Autocorr_Hit=Shotlog_1415.groupby('shoot_player')[['current_shot_hit','lag_shot_hit']].corr().unstack().iloc[:,1].reset_index()
Autocorr_Hit.head()

,shoot_player,current_shot_hit
,,lag_shot_hit
0,aaron brooks,0.007921
1,aaron gordon,-0.094916
2,al farouq aminu,0.062406
3,al horford,0.009093
4,al jefferson,-0.042402


In [ ]:
Autocorr_Hit.columns=Autocorr_Hit.columns.get_level_values(0)
Autocorr_Hit.head()

,shoot_player,current_shot_hit
0,aaron brooks,0.007921
1,aaron gordon,-0.094916
2,al farouq aminu,0.062406
3,al horford,0.009093
4,al jefferson,-0.042402


In [ ]:
Autocorr_Hit.rename(columns={'current_shot_hit':'autocorr'}, inplace=True)
Autocorr_Hit.head()

,shoot_player,autocorr
0,aaron brooks,0.007921
1,aaron gordon,-0.094916
2,al farouq aminu,0.062406
3,al horford,0.009093
4,al jefferson,-0.042402


In [ ]:
Autocorr_Hit.sort_values(by=['autocorr'], ascending=[False]).head(10)

,shoot_player,autocorr
131,joey dorsey,0.190611
143,jusuf nurkic,0.122503
200,nate robinson,0.114813
114,jason smith,0.110956
160,kostas papanikolaou,0.103030
7,alex len,0.101468
169,lavoy allen,0.095943
5,alan anderson,0.089128
35,brian roberts,0.087965
278,zach lavine,0.083653


## Part 3 - Regression Analyses

In [ ]:
#Reg1: Run a linear least squares regression using the entire shotlog dataframe. Include the following control variables:

    #Shot distance

    #Number of dribbles

    #Touch time

    #Type of shot (“points” variable)

    #Quarter of the game (as a categorical variable)

    #Home or away game

    #Shoot_player

    #Closest defender

    #Closest defender distance

In [ ]:
reg1 = sm.ols(formula = 'error ~ lagerror+shot_dist+dribbles+touch_time+points+C(quarter)+home_away+shoot_player+closest_defender+closest_def_dist', data= Shotlog_1415).fit()
print(reg1.summary())


In [ ]:
reg2 = sm.wls(formula = 'error ~ lagerror+shot_dist+dribbles+touch_time+C(points)+C(quarter)+home_away+shoot_player+closest_defender+closest_def_dist',  weights=1/Shotlog_1415['shot_per_game'] , data= Shotlog_1415).fit()
print(reg2.summary())

In [ ]:
Reg3_player=sm.ols(formula = 'error ~ lagerror+shot_dist+dribbles+touch_time+C(points)+C(quarter)+home_away+closest_def_dist', data= Shotlog_player).fit()
print(reg3.summary())

In [ ]:
Reg4_wls_player=sm.wls(formula = 'error ~ lagerror+shot_dist+dribbles+touch_time+C(points)+C(quarter)+home_away+closest_def_dist',weights=1/Shotlog_player['shot_per_game'] , data= Shotlog_player).fit()
print(reg4.summary())